In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

In [6]:

data = pd.read_csv("data.txt", sep='  ', header=None)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [19]:
x = data.iloc[:,:16]
y = data.iloc[:,16:18]
#y2 =  data.iloc[:,17]
x = x.drop(columns = [8,11,0,1,3,5,6,13])

In [20]:
y

,16,17
0,0.95,0.975
1,0.95,0.975
2,0.95,0.975
3,0.95,0.975
4,0.95,0.975
5,0.95,0.975
6,0.95,0.975
7,0.95,0.975
8,0.95,0.975
9,0.95,0.976


In [21]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.3)

In [22]:
def sigmoid(x):
    return 1/(1+ np.exp(-x))

In [23]:
def dsigmoid(x):
    return x*(1-x)

In [33]:
np.random.seed(3) #generates a value between 3
w0 = np.random.random((8,50)) - 1 #between layers 0 and 1
w1 = np.random.random((50,26)) - 1 #between layers 1 and 2
w2 = np.random.random((26,8)) - 1
w3 = np.random.random((8,2)) - 1

b0 = np.random.random((1,1)) - 1
b1 = np.random.random((1,1)) - 1
b2 = np.random.random((1,1)) - 1
b3 = np.random.random((1,1)) - 1

In [34]:
def feed_forward(data_in , w0, w1 ,w2,w3, b0,b1,b2,b3):
    layer0 = data_in
    layer1 = sigmoid(np.dot(layer0,w0)+b0)
    layer2 = sigmoid(np.dot(layer1,w1)+b1)
    layer3 = sigmoid(np.dot(layer2,w2)+b2)
    layer4 = sigmoid(np.dot(layer3,w3)+b3)
    
    return layer0,layer1,layer2,layer3,layer4

In [64]:
def back_propogation(layer0,layer1,layer2,layer3,layer4, actual_y, w0,w1,w2,w3,b0,b1,b2,b3,learning_rate, i):
    
    l4_error = layer4 - actual_y
    l4_delta = l4_error
    dh4 = np.dot(layer3.T,l4_delta)
    
    l3_error = np.dot(l4_delta,w3.T)
    l3_delta = l3_error * dsigmoid(layer3)
    dh3 = np.dot(layer2.T, l3_delta) #layer 1 change
    
    l2_error = np.dot(l3_delta,w2.T)
    l2_delta = l2_error * dsigmoid(layer2)
    dh2 = np.dot(layer1.T,l2_delta)#layer2 changes
    
    l1_error = np.dot(l2_delta,w1.T)
    l1_delta = l1_error * dsigmoid(layer1)
    dh1 = np.dot(layer0.T, l1_delta)#layer 3 changes
    
    w3 = w3 - (learning_rate * dh4)
    w2 = w2 - (learning_rate * dh3)
    w1 = w1 - (learning_rate * dh2)
    w0 = w0 - (learning_rate * dh1)
   # b3 = b3 - (learning_rate * np.mean(l4_delta))
    b2 = b2 - (learning_rate * np.mean(l3_delta))
    b1 = b1 - (learning_rate * np.mean(l2_delta))#we use l2_delta as we get a unit value when differentiation with b is done
    b0 = b0 - (learning_rate * np.mean(l1_delta))
    
    if i%1 == 0 and (i!=0):
        loss = np.abs(layer4 - actual_y)
        loss_curve.append(loss)
        iters.append(int(i))
        
        if i%1000 == 0:
            print('\n', int(i),loss)
    
    return w0,w1,w2,w3,b0,b1,b2,b3
    

In [97]:
epochs = 10000
loss_curve = []
iters = []


In [98]:
for i in range(epochs):
    layer0,layer1,layer2,layer3,layer4 = feed_forward(xtrain,w0,w1,w2,w3,b0,b1,b2,b3)
    w0,w1,w2,w3,b0,b1,b2,b3 = back_propogation(layer0,layer1,layer2,layer3,layer4,ytrain, w0,w1,w2,w3,b0,b1,b2,b3,0.05,i)


 1000           16            17
9077   0.012  4.999569e-03
6346   0.023  2.199957e-02
4856   0.030  5.999569e-03
9206   0.011  1.699957e-02
622    0.048  7.999569e-03
8602   0.014  5.999569e-03
8858   0.013  2.999569e-03
10838  0.004  1.699957e-02
851    0.047  8.999569e-03
4427   0.032  1.999569e-03
2937   0.038  1.099957e-02
5227   0.028  1.699957e-02
2534   0.040  3.999569e-03
10868  0.004  1.399957e-02
9440   0.010  1.699957e-02
5960   0.025  1.299957e-02
9789   0.009  3.999569e-03
322    0.049  1.599957e-02
3538   0.035  2.199957e-02
6136   0.024  1.999957e-02
11153  0.003  7.999569e-03
9745   0.009  8.999569e-03
5352   0.028  2.999569e-03
3526   0.035  2.399957e-02
6649   0.022  1.499957e-02
2297   0.041  3.999569e-03
3945   0.034  2.999569e-03
11326  0.002  1.499957e-02
4817   0.030  9.999569e-03
5534   0.027  8.999569e-03
...      ...           ...
2725   0.039  8.999569e-03
7234   0.020  1.999569e-03
2673   0.039  1.399957e-02
4203   0.033  4.306167e-07
7302   0.019  1.99995


 7000           16     17
9077   0.012  0.005
6346   0.023  0.022
4856   0.030  0.006
9206   0.011  0.017
622    0.048  0.008
8602   0.014  0.006
8858   0.013  0.003
10838  0.004  0.017
851    0.047  0.009
4427   0.032  0.002
2937   0.038  0.011
5227   0.028  0.017
2534   0.040  0.004
10868  0.004  0.014
9440   0.010  0.017
5960   0.025  0.013
9789   0.009  0.004
322    0.049  0.016
3538   0.035  0.022
6136   0.024  0.020
11153  0.003  0.008
9745   0.009  0.009
5352   0.028  0.003
3526   0.035  0.024
6649   0.022  0.015
2297   0.041  0.004
3945   0.034  0.003
11326  0.002  0.015
4817   0.030  0.010
5534   0.027  0.009
...      ...    ...
2725   0.039  0.009
7234   0.020  0.002
2673   0.039  0.014
4203   0.033  0.000
7302   0.019  0.020
6848   0.021  0.019
6033   0.025  0.005
351    0.049  0.012
3823   0.034  0.017
2211   0.041  0.014
4553   0.031  0.014
3323   0.036  0.020
6902   0.021  0.013
2754   0.039  0.005
8768   0.013  0.013
7797   0.017  0.017
3342   0.036  0.018
10332  0.006 

In [74]:
#plt.plot(iters,loss_curve)

In [115]:
c = feed_forward(xtest.iloc[300:400,:],w0,w1,w2,w3,b0,b1,b2,b3)
c[4]

array([[1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.        , 0.99999999],
       [1.

In [105]:
ytest.iloc[250:251,:]

,16,17
630,0.952,0.993
